In [ ]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed) push test
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(dataset[0])


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch

# Parameters
batch_size = 2
num_negatives = 2
num_users = 3
num_items = 3
dim = 2

# Embeddings for users and items (3 users, 3 items, embedding dimension 2)
user_embedding = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
item_embedding = torch.tensor([[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]])

# h_index and t_index based on the edges and negative samples
# Each row represents a batch entry, and each column is a negative sample
# Here, `0` and `1` are valid head indices (users), and we corrupt them by keeping valid and invalid tails
h_index = torch.tensor([
    [0, 0, 0],  # For first edge (0, 0, 3) with two negatives
    [1, 2, 0]   # For second edge (1, 0, 4) with two negatives
])
t_index = torch.tensor([
    [3, 4, 5],  # Original (0, 0, 3), and corrupted tails [4, 2]
    [4, 4, 4]   # Original (1, 0, 4), and corrupted tails [5, 0]
])

# Gather head node embeddings
# (num_nodes, dim)
index_temp = h_index.unsqueeze(-1).expand(-1, -1, user_embedding.shape[-1])
h_embeddings = user_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, h_index.unsqueeze(-1).expand(-1, -1, dim))

# Adjust `t_index` to map to item IDs by subtracting `num_users`
index_temp = (t_index - num_users).clamp(min=0)
t_embeddings = item_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, index_temp.unsqueeze(-1).expand(-1, -1, dim))

print("User Embedding Tensor:\n", user_embedding)
print("Item Embedding Tensor:\n", item_embedding)
print("Head Embeddings:\n", h_embeddings)
print("Tail Embeddings:\n", t_embeddings)
print("size head Embeddings:\n", h_embeddings.shape)
print("size Tail Embeddings:\n", t_embeddings.shape)


User Embedding Tensor:
 tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]])
Item Embedding Tensor:
 tensor([[0.7000, 0.8000],
        [0.9000, 1.0000],
        [1.1000, 1.2000]])
Head Embeddings:
 tensor([[[0.1000, 0.2000],
         [0.1000, 0.2000],
         [0.1000, 0.2000]],

        [[0.3000, 0.4000],
         [0.5000, 0.6000],
         [0.1000, 0.2000]]])
Tail Embeddings:
 tensor([[[0.7000, 0.8000],
         [0.9000, 1.0000],
         [1.1000, 1.2000]],

        [[0.9000, 1.0000],
         [0.9000, 1.0000],
         [0.9000, 1.0000]]])
size head Embeddings:
 torch.Size([2, 3, 2])
size Tail Embeddings:
 torch.Size([2, 3, 2])


In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [9]:
import torch
t_ranking = torch.zeros(5)
h_ranking = torch.zeros(6)
rankings1 = [t_ranking, h_ranking]
print(rankings1)
rankings2 = torch.cat([t_ranking, h_ranking], dim=0)
print(rankings2)


[tensor([0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0.])]
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [7]:
import optuna
import sqlite3

# Enable WAL mode
conn = sqlite3.connect("optuna_hyperparam_tuning.db")
conn.execute("PRAGMA journal_mode=WAL;")
conn.close()
print ("done")


done


In [2]:
import optuna
#from Ultra import util

storage_url = "sqlite:////itet-stor/trachsele/net_scratch/tl4rec/optuna_hyperparam_tuning.db"
#optuna.delete_study(study_name="my_hyperparam_study", storage=storage_url)
#print ("deleted")
# Load the study
study = optuna.load_study(study_name="my_hyperparam_study", storage=storage_url)

# Get the best trial
best_trial = study.best_trial

best_params = best_trial.params

#util.save_cfg_of_best_params(best_params, cfg)
#logger.warning("hyperparam search done and cfg saved")

# Print the best parameters
print("Best Parameters:", best_trial.params)
print("Best Value:", best_trial.value)

Best Parameters: {'projection_edge_lr': 0.000248786603430893, 'backbone_conv_lr': 0.00017783663799881164, 'backbone_mlp_edge_lr': 0.0003360904636420548, 'edge_projection_use_dropout': True, 'edge_projection_dropout_rate': 0.3743907806104132, 'edge_projection_use_layer_norm': False, 'num_edge_proj_layers': 2, 'edge_emb_dim': 16, 'embedding_edge_use_dropout': True, 'embedding_edge_dropout_rate': 0.03247182903513618, 'embedding_edge_use_layer_norm': False, 'num_edge_emb_layers': 6, 'simple_model_dim': 32, 'simple_model_num_hidden': 7, 'num_negative': 54, 'adversarial_temperature': 1.0}
Best Value: 0.23600001633167267


In [ ]:
%run script/pretrain.py -c config/recommender/pretrain_all.yaml --gpus [0]

In [1]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset LastFM --epochs 1 --bpe 10000 --gpus "[0]" --ckpt /itet-stor/trachsele/net_scratch/tl4rec/ckpts/Beauty_Games_4_40_proj.pth

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
16:12:02   Random seed: 1024
16:12:02   Config file: config/recommender/notebook_cfg.yaml
16:12:02   {'checkpoint': '/itet-stor/trachsele/net_scratch/tl4rec/ckpts/Beauty_Games_4_40_proj.pth',
 'dataset': {'class': 'LastFM',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_edge': {'dropout_rate': 0.2,
                                                 'hidden_dims': [4, 4, 4],
                                                 'use_dropout': True,
                                                 'use_layer_norm': True},
                              'embedding_item': {'dropout_rate': 0.1,
                                              

We will evaluate vs 1000 negatives
bpe 9930.0
discarded user_features
edge_attr.shape = torch.Size([79442, 1])


16:12:02   ------------------------------
16:12:02   Number of parameters: 53941
16:12:02   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:12:02   Epoch 0 begin


freeze my backbone
unfreeze my backbone
Load rspmm extension. This may take a while...


16:13:33   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:13:33   binary cross entropy: 0.691503
16:13:33   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:13:33   Epoch 0 end
16:13:33   ------------------------------
16:13:33   average loss: 0.696634
16:13:33   Save checkpoint to model_epoch_1.pth
16:13:33   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:13:33   Evaluate on valid
16:13:54   mr: 73.2225
16:13:54   mrr: 0.0204713
16:13:54   hits@1: 0.000562482
16:13:54   hits@3: 0.00187494
16:13:54   hits@10: 0.0149995
16:13:54   ndcg@20: 0.0167062
16:13:54   Load checkpoint from model_epoch_1.pth
16:13:55   Save final_ckpt to /itet-stor/trachsele/net_scratch/tl4rec/ckpts/pretrain/LastFM.pth
16:13:55   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:13:55   Evaluate on valid
16:14:22   mr: 740.975
16:14:22   mrr: 0.00205837
16:14:22   hits@1: 0
16:14:22   hits@3: 9.37471e-05
16:14:22   hits@10: 0.000281241
16:14:22   ndcg@20: 0.000444652
16:14:22   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:14:22   Evaluate on test
16:14:55   mr: 350.16
16:14:55   

In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Yelp18 --epochs 10 --bpe 20000 --gpus "[0]" --ckpt null

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
16:22:10   Random seed: 1024
16:22:10   Config file: config/recommender/notebook_cfg.yaml
16:22:10   {'checkpoint': None,
 'dataset': {'class': 'Yelp18',
             'root': '/itet-stor/trachsele/net_scratch/tl4rec/model_outputs/data'},
 'model': {'backbone_model': {'embedding_edge': {'dropout_rate': 0.03247182903513618,
                                                 'hidden_dims': [16,
                                                                 16,
                                                                 16,
                                                                 16,
                                                                 16,
                                            

We will evaluate vs all negatives


16:22:11   Yelp18 dataset
16:22:11   #train: 1097007, #valid: 140252, #test: 324147


bpe 13712.0
discarded node_features
edge_attr.shape = torch.Size([2194014, 5])


16:22:11   ------------------------------
16:22:11   Number of parameters: 86865
16:22:11   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:22:11   Epoch 0 begin


Load rspmm extension. This may take a while...


16:22:13   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:22:13   binary cross entropy: 1.52934
16:22:54   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:22:54   binary cross entropy: 0.526128
16:23:36   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:23:36   binary cross entropy: 0.491356
16:24:18   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:24:18   binary cross entropy: 0.365914


In [ ]:
%run script/run.py -c config/recommender/notebook_cfg.yaml --dataset Epinions --epochs 1 --bpe 10 --gpus "[0]" --ckpt null

In [ ]:
import torch
from rspmm import generalized_rspmm

# Test input data
edge_index = torch.tensor([[0, 1, 2], [1, 2, 3]], dtype=torch.long)
edge_type = torch.tensor([0, 1, 2], dtype=torch.long)
edge_weight = torch.tensor([1.0, 1.0, 1.0], dtype=torch.float32)
edge_attr = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]], dtype=torch.float32)  # Edge features
relation = torch.rand(3, 4)  # Random relation features
input_features = torch.rand(4, 4)  # Node input features

# Call the function
output = generalized_rspmm(edge_index, edge_type, edge_weight, edge_attr, relation, input_features)
print("Output:", output)


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [7]:
import pickle


def inspect_cxtdict(cxtdict):
    print("Type:", type(cxtdict))
    print("Number of entries:", len(cxtdict))
    if isinstance(cxtdict, dict):
        print("Number of entries:", len(cxtdict))
        # Print a sample of the keys and values
        for i, key in enumerate(cxtdict.keys()):
            print(f"Key {i}: {key}, Value: {cxtdict[key]}")
            if i >= 9:  # Limit to first 10 entries
                break


def load_data(filename):
    try:
        with open(filename, "rb") as f:
            x= pickle.load(f)
    except:
        x = []
    return x
    
CXTDict = load_data('/itet-stor/trachsele/net_scratch/tl4rec/CARCA_Code_and_Data/Data/CXTDictSasRec_Games.dat')
inspect_cxtdict(CXTDict)
#print(CXTDict)
#print(CXTDict[(1,49583)])



Type: <class 'dict'>
Number of entries: 287107
Number of entries: 287107
Key 0: (352, 60), Value: [0.0, 0.5, 0.7419354838709677, 0.2857142857142857, 0.4767123287671233, 6.25]
Key 1: (1814, 647), Value: [0.0, 0.8333333333333334, 0.45161290322580644, 0.42857142857142855, 0.7863013698630137, 10.25]
Key 2: (1796, 619), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 3: (1504, 173), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 4: (1504, 364), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 5: (1504, 592), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 6: (1504, 785), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.8602739726027397, 11.25]
Key 7: (1504, 797), Value: [0.0, 0.9166666666666666, 0.3225806451612903, 0.2857142857142857, 0.86027397

In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pitri (pitri-eth-z-rich). Use `wandb login --relogin` to force relogin


acc,▂▁▇▆▆▇█▇
loss,█▃▃▃▄▃▃▁
acc,0.83106
loss,0.0692
